In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import multivariate_normal

## Gaussian Mixture Models

### K-Means Algorithm

In [13]:
class KMeans_clustering:
    Data = []
    K = 0 #Number of Clusters
    E_mean = 0
    
    def __init__(self,Data,number_of_clusters):
        self.Data = Data
        self.K = number_of_clusters

        epsilon_constant = 0.1
        self.E_mean = [] #np.ones((self.K,np.shape(self.Data[0]))) * (0.01)  # This means that less than when change in means is less than 0.01 the process stops
        for i in range(self.K):
            self.E_mean.append(np.zeros(np.shape(self.Data[0])))

        self.E_mean = np.array(self.E_mean)
            
            

    def cluster_allocator(self,new_means_array):
        DATA_COPY = np.copy(self.Data)
        cluster_array = []

        for i in range(self.K):
            cluster_array.append([])
        
        for i in range(len(DATA_COPY)):
            #distance calculation of points from means
            distance_vectors = new_means_array - DATA_COPY[i]
            distances_array = []

            for j in distance_vectors:
                #print(j)
                j = np.matrix(j)
                #distances_array.append(np.sqrt(j@j))
                distances_array.append(np.matmul(j,np.transpose(j)))
            
            distances_array = np.array(distances_array)

            min_distance_index = np.argmin(distances_array)
            cluster_array[min_distance_index].append(i)

        return cluster_array

    def centroid_calculator(self,cluster_array):
        new_means_array = []

        for i in cluster_array:
            # i is an array of indices
            total_vector = 0
            mean_vector = 0
            for j in i:
                #j are indices from the main data array
                total_vector += self.Data[j]

            mean_vector = total_vector/len(i)
            new_means_array.append(mean_vector)

        new_means_array = np.array(new_means_array)
        return new_means_array

                
    def cluster_array_decoder(self,cluster_array):
        cluster_array_decoded = []

        for i in cluster_array:
            #we get an array of indices
            array_1 = []
            for j in i:
                #Now we get indices
                array_1.append(self.Data[j])

            cluster_array_decoded.append(array_1)

        return cluster_array_decoded
                
    
    def main_process_single_iteration(self,iterator_limit):
        #print('dsfsdfdsd')
        DATA = np.copy(self.Data)
        new_means_array = [] #np.zeros((self.K,len(DATA[0])))

        #initialising means randomly
        for i in range(self.K):
            random_number = np.random.randint(0,len(DATA))
            new_means_array.append(DATA[random_number])
            DATA = np.delete(DATA,random_number,axis=0)

        #print(new_means_array)
        new_means_array = np.array(new_means_array)
        DATA = np.copy(self.Data)

        
        old_means_array = np.zeros_like(new_means_array) 
        #Now the loop starts
        iterator = 1
 
        while True and iterator <= iterator_limit:
            #print(iterator)
            #Cluster Allocation Step
            cluster_array = self.cluster_allocator(new_means_array)
            old_means_array = np.copy(new_means_array)

            #new means calculation step
            new_means_array = self.centroid_calculator(cluster_array)

            #difference between old and new array
            mean_difference_array = np.abs(new_means_array - old_means_array)#new_means_array - old_means_array

            #checking termination condition
            number_of_means_satisfying_termination_criteria = np.count_nonzero(mean_difference_array < self.E_mean)

            if number_of_means_satisfying_termination_criteria == self.K:
                break

            iterator += 1

        return new_means_array,cluster_array

    def WCSS_calculator(self,means_array,cluster_array):  #Within clusters sum of squares
        WCSS = 0

        decoded_cluster_array = self.cluster_array_decoder(cluster_array)

        for count,i in enumerate(decoded_cluster_array):
            for j in i:
                difference_vector_from_allocated_mean = np.matrix(j - means_array[count])
                WCSS += np.matmul(difference_vector_from_allocated_mean,np.transpose(difference_vector_from_allocated_mean))

        return WCSS
        
    def find_optimal_cluster(self,N,iterator_limit):
        
        cluster_new = np.array([])
        means_new = np.array([])
        WCSS_new = 0

        means_array_old,cluster_array_old = self.main_process_single_iteration(iterator_limit)
        WCSS_old = self.WCSS_calculator(means_array_old,cluster_array_old)
        
        i=0
        while i<N:
            #print(WCSS_old)
            means_array_new,cluster_array_new = self.main_process_single_iteration(iterator_limit)
            WCSS_new = self.WCSS_calculator(means_array_new,cluster_array_new)

            if WCSS_new < WCSS_old:
                WCSS_old = WCSS_new
                means_array_old = means_array_new

                #print(cluster_array_new)
                cluster_array_old = cluster_array_new

            elif WCSS_new >= WCSS_old:
                #Do Nothning
                pass
            
            i += 1

        return means_array_old,self.cluster_array_decoder(cluster_array_old)
            

### Parameter estimation using Expectation Maximisation Algorithm

In [14]:
class Expectation_Maximisation_FOR_GMM_Params:
    DATA = []
    number_of_Gaussians = 0
    threshold = 0
    theta = []  #parameter vector as per notes
    N = 0 #Number of Data points
    diag_cov_flag = 0

    def __init__(self,Data,number_of_Gaussians,threshold=0.001,diagonal_cov_mat=0):
        self.diag_cov_flag = diagonal_cov_mat
        self.DATA = Data
        self.number_of_Gaussians = number_of_Gaussians
        #super().__init__(Data,number_of_Gaussians)
        self.theta = []
        self.N = np.shape(Data)[0]
        self.threshold = threshold

    def Intialisation_step(self):   #Do K-Means-clustering
        w_array = []
        means_array = []
        covariance_matrix_array = []
        Nq_array = []  #number of points in qth cluster/gaussian
        
        Q = self.number_of_Gaussians

        OBJECT_1 = KMeans_clustering(self.DATA,self.number_of_Gaussians)
        KMEANS_means_array,KMEANS_cluster_array = OBJECT_1.find_optimal_cluster(3,15) #3,50

        for q in range(len(KMEANS_cluster_array)):
            KMEANS_cluster_array[q] = np.array(KMEANS_cluster_array[q])

        for q in range(Q):
            N_q = np.shape(KMEANS_cluster_array[q])[0]
            w_q = N_q/self.N
            w_array.append(w_q)
            
            cluster_mean = KMEANS_means_array[q]
            means_array.append(cluster_mean)

            #calculating the covariance matrix
            n_features = np.shape(self.DATA)[1]
            cov_matrix = np.zeros((n_features,n_features))

            '''
            for i in KMEANS_cluster_array[q]:
                difference_from_mean = np.matrix(i - cluster_mean)
                cov_matrix += (np.matmul(difference_from_mean.T,difference_from_mean))/N_q
            '''
##############################################
            

            if self.diag_cov_flag==1:
                shape_cov_mat = np.shape(np.cov(KMEANS_cluster_array[q].T))
                toappend = np.multiply(np.cov(KMEANS_cluster_array[q].T),np.eye(shape_cov_mat[0]))
                covariance_matrix_array.append(toappend)

            else:
                covariance_matrix_array.append(np.cov(KMEANS_cluster_array[q].T))
                
##############################################

        theta_old = [w_array,means_array,covariance_matrix_array] #as per notes

        return theta_old    #,KMEANS_means_array,KMEANS_cluster_array


    def NORMAL_DISTRIBUTION(self,mu,cov_mat,Point):
        inv_cov_mat = np.linalg.inv(cov_mat)
        difference_from_mean = np.matrix(Point - mu)
        d = np.shape(Point)[0]
        #print(d)

        exponentail_term_power = -(np.matmul(np.matmul(difference_from_mean,inv_cov_mat),difference_from_mean.T))/2

        rational_term_denominator = np.sqrt(((2*np.pi)**d) * np.linalg.det(cov_mat))
        #print(rational_term_denominator)

        Normal_prob = np.exp(exponentail_term_power[0,0])/(rational_term_denominator) # + 1)  #regularisation
        return Normal_prob
        
        
    def Expectation_step_E(self,theta_old): #KMEANS_means_array,KMEANS_cluster_array):
        responsibility_array = np.zeros((self.number_of_Gaussians,self.N))  #structured as per Cluster array

        responsibility_numerator_array = np.zeros_like(responsibility_array) #structured as per Cluster array
        responsibility_denominator_array = np.zeros(np.shape(self.DATA)[0])
        numerator_sum = 0.0

        w_array = theta_old[0]
        means_array = theta_old[1]
        covariance_matrix_array = theta_old[2]
        
        
        for n,i in enumerate(self.DATA):
            for q in range(self.number_of_Gaussians):
                responsibility_denominator_array[n] += w_array[q] * multivariate_normal.pdf(i,means_array[q],covariance_matrix_array[q])
                responsibility_numerator_array[q][n] = w_array[q] * multivariate_normal.pdf(i,means_array[q],covariance_matrix_array[q])
        
        for q in range(self.number_of_Gaussians):
            for n,i in enumerate(self.DATA):
                responsibility_array[q][n] = responsibility_numerator_array[q][n]/responsibility_denominator_array[n]
                #print(responsibility_numerator_array[q][n])
                #numerator_sum += responsibility_numerator_array[q][n]

        #responsibility_array = responsibility_numerator_array/numerator_sum

        return responsibility_array

        '''
        for q in range(self.number_of_Gaussians):
            qth_row_in_resp_vector = []
            cluster_list = KMEANS_cluster_array[q]
            wq = theta_old[0][q]
            gaussian_mean = theta_old[1][q] #OR KMEANS_means_array[q]
            cov_mat = theta_old[2][q]

            for j in cluster_list:
                gamma_nq = wq * self.NORMAL_DISTRIBUTION(gaussian_mean,cov_mat,j)
                numerator_sum += gamma_nq
                qth_row_in_resp_vector.append(gamma_nq)

            responsibility_numerator_array.append(qth_row_in_resp_vector)

        for i in responsibility_numerator_array:
            for j in i:
                responsibility_array.append(j/numerator_sum)

        return responsibility_array
        '''


    def Maximisation_step_M(self,responsibility_array,theta_old): #,KMEANS_cluster_array):
        Q = self.number_of_Gaussians
        N_q_array = np.zeros(Q)

        #Calculating N_q
        for q in range(Q):
            for n,i in enumerate(self.DATA):
                N_q_array[q] += responsibility_array[q][n]

        #Calculating means mu
        means_array = []
        for q in range(Q):
            qth_mean = 0.0
            for n,i in enumerate(self.DATA):
                qth_mean += (responsibility_array[q][n] * i)/N_q_array[q]

            means_array.append(qth_mean)


        #Calcuting covariance matrix array
        covariance_matrix_array = []
        for q in range(Q):
            qth_cov_mat = np.zeros_like(theta_old[2][0])
            qth_mean = means_array[q]
            N_q = N_q_array[q]

            for n,i in enumerate(self.DATA):
                difference_from_mean = np.matrix(i - qth_mean)
                matrix_term = np.matmul(difference_from_mean.T,difference_from_mean)
                qth_cov_mat += (responsibility_array[q][n] * matrix_term)/N_q
####################################
            # covariance_matrix_array.append(qth_cov_mat)
            if self.diag_cov_flag==1:
                shape_cov_mat = np.shape(qth_cov_mat)
                toappend = np.multiply(qth_cov_mat,np.eye(shape_cov_mat[0]))
                covariance_matrix_array.append(toappend)

            else:
                covariance_matrix_array.append(qth_cov_mat)
                 
                
####################################
        #Calculating w_q
        w_array = []

        for q in range(Q):
            N_q = N_q_array[q]
            w_q = N_q/self.N
            w_array.append(w_q)


        theta_new = [w_array,means_array,covariance_matrix_array]
        return theta_new

        '''
        #calculating Nq
        for i in responsibility_array:
            N_q = np.sum(i)
            N_q_array.append(N_q)


        #calculating means mu
        means_array = []
        
        for q in range(Q):
            N_q = N_q_array[q]
            qth_mean = 0

            for i,j in zip(responsibility_array[q],KMEANS_cluster_array[q]):
                qth_mean += (i*j)/N_q

            means_array.append(qth_mean)


        #calculating covaricnce matrices
        covariance_matrix_array = []

        for q in range(Q):
            N_q = N_q_array[q]
            qth_cov_mat = np.zeros_like(theta_old[2][0])
            qth_mean = means_array[q]

            for i,j in zip(responsibility_array[q],KMEANS_cluster_array[q]):
                difference_from_mean = np.matrix(j - qth_mean)
                matrix_term = np.matmul(difference_from_mean.T,difference_from_mean)
                qth_cov_mat += (i * matrix_term)/N_q

            covariance_matrix_array.append(qth_cov_mat)


        #calculating wq
        w_array = []

        for q in range(Q):
            N_q = N_q_array[q]
            w_q = N_q/self.N
            w_array.append(w_q)

        theta_new = [w_array,means_array,covariance_matrix_array]
        return theta_new
        '''

    
    def LOG_LIKELIHOOD_CALCULATOR(self,theta):
        w_array = theta[0]
        means_array = theta[1]
        covariance_matrix_array = theta[2]

        #CHECK SLIDES FOR LOG LIKELIHOOD FUNCTION
        log_term = 0.0

        for i in self.DATA:
            term_inside_log = 0.0
            for q in range(self.number_of_Gaussians):
                term_inside_log += w_array[q] * multivariate_normal.pdf(i,means_array[q],covariance_matrix_array[q])#self.NORMAL_DISTRIBUTION(means_array[q],covariance_matrix_array[q],i)
                #print('sdsdsf',self.NORMAL_DISTRIBUTION(means_array[q],covariance_matrix_array[q],i))   
            #print('term_inside log',term_inside_log)
            log_term += np.log(term_inside_log)

        log_likelihood = log_term
        return log_likelihood

    
    def likelihood_difference_step(self,theta_new,theta_old):
        LL_OLD = self.LOG_LIKELIHOOD_CALCULATOR(theta_old)
        LL_NEW = self.LOG_LIKELIHOOD_CALCULATOR(theta_new)

        #print(LL_OLD,LL_NEW)

        difference = LL_NEW - LL_OLD
        print(difference)
        return difference

    

    '''
    def LOG_LIKELIHOOD_CALCULATOR(self, theta):
        w_array, means_array, covariance_matrix_array = theta
    
        log_term = 0.0
    
        for i in self.DATA:
            term_inside_log = 0.0
            for q in range(self.number_of_Gaussians):
                term_inside_log += w_array[q] * self.NORMAL_DISTRIBUTION(means_array[q], covariance_matrix_array[q], i)
            log_term += np.log1p(term_inside_log)  # Use np.log1p to handle small numbers accurately
    
        log_likelihood = log_term
        return log_likelihood

    def likelihood_difference_step(self, theta_new, theta_old):
        LL_OLD = self.LOG_LIKELIHOOD_CALCULATOR(theta_old)
        LL_NEW = self.LOG_LIKELIHOOD_CALCULATOR(theta_new)
    
        difference = LL_NEW - LL_OLD
        return difference
    '''



    def main_process(self,max_iterator_count):  #max_iterator_count means process will stop after that value if no convergence takes place
        iterator = 1
        #INITIALISATION STEP
        theta_old = self.Intialisation_step()

        while iterator <= max_iterator_count:
            print('iterator=',iterator)
            
            #EXPECTATION STEP
            responsibility_array = self.Expectation_step_E(theta_old)

            #MAXIMISATION STEP
            theta_new = self.Maximisation_step_M(responsibility_array,theta_old)

            #COMPARISON STEP
            difference = self.likelihood_difference_step(theta_new,theta_old)
            #print(difference)

            if np.abs(difference) <= self.threshold:
             #   print('lauda lelo')
                print('CONVERGED')
                break

            elif np.abs(difference) > self.threshold:    
                theta_old = theta_new.copy()
                #theta_old = self.Intialisation_step()

            iterator += 1

        return theta_new

                

### GMM-based Classifier

In [15]:
class GMM_Based_Bayes_classifier:
    Training_data = []
    Training_labels = []
    label_array = []

    #Test_data = []
    #Test_labels = []

    Training_data_classwise = []
    number_of_gaussians = [] #Q

    #parameters_array = []
    N = 0

    # def extract_diagonal(self,matrix):
    #     new_matrix = np.zeros_like(matrix)
    #     ROWS = np.shape(new_matrix)[0]
    #     COLUMNS = np.shape(new_matrix)[1]
    #     matrix = np.matrix(matrix)
    #     diagonal_vector = np.array(matrix.diagonal())

    #     for i in range(ROWS):
    #         for j in range(COLUMNS):
    #             if i==j:
    #                 new_matrix[i][i] = diagonal_vector[i]

    #     return new_matrix
        
    def __init__(self,TrD,TrL,Q,diagonal_cov_mat=0):  #diagonal_cov_mat flag
        self.Training_data = TrD
        self.Training_labels = TrL

        self.label_array = np.unique(TrL)
        
        #self.Test_data = TeD
        #self.Test_labels = TeL

        self.number_of_gaussians = Q
        self.N = np.shape(TrD)[0]

        #Parameter estimation step

        #classwise _ training data
        n_classes = np.shape(np.unique(TrL))[0]

        Training_data_classwise = []
        for i in range(n_classes):
            Training_data_classwise.append([])

        for index,data_point in enumerate(TrD):
            Training_data_classwise[int(TrL[index])].append(data_point)

        self.Training_data_classwise = Training_data_classwise

        #Parameter_estimation_step
        parameter_array = []
        for i in range(n_classes):
            parameter_array.append([])

        for i in range(n_classes):
            PARAM_EST_OBJECT = Expectation_Maximisation_FOR_GMM_Params(self.Training_data_classwise[i],self.number_of_gaussians,0.001,diagonal_cov_mat)
            theta = PARAM_EST_OBJECT.main_process(110)  #param is number of steps before process stops
            
            '''
            if diagonal_cov_mat==1:
                for j in range(Q):
                    #theta[j] is a covariance matrix
                    theta[j][2] = np.multiply(theta[j][2],np.eye(np.shape(theta[j][2])[0]))
            '''
            

            parameter_array[i] = theta

        
        self.parameters_array = parameter_array
        #print(parameter_array)

    
    def NORMAL_DISTRIBUTION(self,mu,cov_mat,Point):
        inv_cov_mat = np.linalg.inv(cov_mat)
        difference_from_mean = np.matrix(Point - mu)
        d = np.shape(Point)[0]
        #print(d)

        exponentail_term_power = -(np.matmul(np.matmul(difference_from_mean,inv_cov_mat),difference_from_mean.T))/2

        rational_term_denominator = np.sqrt(((2*np.pi)**d) * np.linalg.det(cov_mat))
        #print(rational_term_denominator)

        Normal_prob = np.exp(exponentail_term_power[0,0])/(rational_term_denominator) #regulariation
        return Normal_prob
    
    
    def LIKELIHOOD_CALCULATOR(self,theta,point):
        w_array = theta[0]
        means_array = theta[1]
        cov_mat_array = theta[2]

        LIKELIHOOD = 0.0

        for i in range(self.number_of_gaussians): #number_of_gaussians = Q
            Prob_from_gaussian_at_i = w_array[i]* multivariate_normal.pdf(point,means_array[i],cov_mat_array[i])
            LIKELIHOOD += Prob_from_gaussian_at_i

        return LIKELIHOOD
            

        
        
    def Posterioir_Probability_calculator(self,test_point):
        n_classes =  np.shape(np.unique(self.Training_labels))[0]
        Posterioir_prob = np.zeros(n_classes)

        #Now doing fpr each class
        for i in range(n_classes):
            P_y_i = (np.shape(self.Training_data_classwise[i])[0])/self.N #Prior Probability

            #Calculating likelihood
            theta = self.parameters_array[i]

            LIKELIHOOD = self.LIKELIHOOD_CALCULATOR(theta,test_point)#self.LIKELIHOOD_CALCULATOR(theta,test_point)

            POSTERIOIR_PROB_i = LIKELIHOOD*P_y_i
            Posterioir_prob[i] = POSTERIOIR_PROB_i

        return Posterioir_prob

    def single_point_classifier(self,test_point):
        Posterioir_prob_array = self.Posterioir_Probability_calculator(test_point)
        max_posterioir_prob_index = np.argmax(Posterioir_prob_array)

        #print(Posterioir_prob_array)

        classified_label = self.label_array[max_posterioir_prob_index]
        return classified_label

    def classifier_on_test_data(self,TEST_DATA):
        classfication_labels = []
    
        index = 0 #############3
        for i in TEST_DATA:
            #print(index) ##############33
            classified_label = self.single_point_classifier(i)
            classfication_labels.append(classified_label)
            index +=1
    
        return classfication_labels
        
        

In [ ]:
row = 0
column = 0
K = 0

def initialize():

    global mean, weight, deviation
    mean = np.zeros(shape=(K, row, column, 3))
    weight = np.ones(shape=(K, row, column)) / K
    deviation = np.ones(shape=(K, row, column))
    
def check():

    global mask_divide, idx
    T = 0.7
    ratio = -1*(weight / deviation)
    idx = ratio.argsort(axis=0)
    ratio.sort(axis=0)
    ratio *= -1
    cum = np.cumsum(ratio, axis=0)
    mask_divide = (cum < T)
    mask_divide = np.choose(idx, mask_divide)
    # mask_divide = np.choose(idx, mask_divide)

def mahalanobis_probability(video):

    global mask_distance, prob

    temp = np.subtract(video, mean)
    temp = np.sum(temp**2, axis=3) / (deviation**2)

    prob = np.exp(temp/(-2)) / (np.sqrt((2*np.pi)**3)*deviation)

    temp = np.sqrt(temp)
    mask_distance = (temp < 2.5*deviation)

def update(video):
    
    global weight, mask_distance, mean, deviation, prob, mask_some

    alpha = 0.2
    rho = alpha * prob
    
    mask_some = np.bitwise_or.reduce(mask_distance, axis=0)
    mask_update = np.where(mask_some == True, mask_distance, -1)

    weight = np.where(mask_update == 1, (1 - alpha) * weight + alpha, weight)
    weight = np.where(mask_update == 0, (1 - alpha) * weight, weight)
    #weight = np.where(mask_update == -1, 0.0001, weight)

    data = np.stack([video]*K, axis=0)
    mask = np.stack([mask_update]*3, axis=3)
    r = np.stack([rho]*3, axis=3)
    
    mean = np.where(mask == 1, (1 - r) * mean + r * data, mean)
    mean = np.where(mask == -1, data, mean)
    
    deviation = np.where(mask_update == 1, np.sqrt((1-rho)*(deviation**2) + rho*(np.sum(np.subtract(video, mean)**2, axis=3))), deviation)
    deviation = np.where(mask_update == -1, 3 + np.ones(shape=(K, row, column)), deviation)


def result(video):

    background = np.zeros(shape=(row, column, 3), dtype=np.uint8)
    foreground = 255 + np.zeros(shape=(row, column, 3), dtype=np.uint8)
    m = np.stack([mask_some]*3, axis=2)
    res = np.where(m == False, foreground, background)

    n = np.bitwise_and(mask_divide, mask_distance)
    n = np.bitwise_or.reduce(n, axis=0)
    n = np.stack([n]*3, axis=2)
    res = np.where(n == True, background, foreground)

    # cv2.imshow('frame', video)
    cv2.imshow('res', res)

def frame_processing(video):
    
    check()
    mahalanobis_probability(video)
    update(video)
    result(video)

def main():

    global row, column, K
    cap = cv2.VideoCapture('vtest.avi')
    #cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    K = 3
    row, column, _ = frame.shape
    initialize()
    fgbg = cv2.createBackgroundSubtractorMOG2(False)
    while(1):
        ret, frame = cap.read()
        #frame = 128 + np.zeros(shape=(row, column, 3), dtype=np.uint8)
        frame_processing(frame)
        cv2.imshow('frame', frame)
        fgmask = fgbg.apply(frame)
        cv2.imshow('fgmask', fgmask)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

main()